### Подгрузка библиотек

In [1]:
import os
os.chdir(os.path.dirname(os.getcwd()))

import pandas as pd

import datetime
from dateutil.relativedelta import relativedelta 

### Считывание данных

In [2]:
data_path = "./data/fact_train_test.csv"
data = pd.read_csv(data_path, sep=";", decimal=",", encoding="windows-1251")
data

,period,rps,podrod,filial,client_sap_id,freight_id,sender_station_id,recipient_station_id,sender_organisation_id,real_weight,real_wagon_count
0,2012-07-01,1,5,1,328,1193,30252,13005,10036,71.0,1
1,2012-10-01,1,5,1,328,1193,30252,11376,10036,210.0,3
2,2014-03-01,0,1,1,328,3472,30252,29548,10036,67.0,1
3,2014-03-01,0,1,1,328,3472,30252,29158,10036,67.0,1
4,2014-03-01,0,2,1,328,3472,30252,27484,10036,66.0,1
...,...,...,...,...,...,...,...,...,...,...,...
3559227,2023-03-01,1,5,2,1346,1482,31438,31482,0,69.0,1
3559228,2023-03-01,1,5,2,1346,1492,31438,36091,27275,70.0,1
3559229,2023-03-01,1,5,2,1346,1492,31438,35450,27275,70.0,1
3559230,2023-03-01,1,5,2,1346,1492,31438,31482,0,207.0,3


### Решение

**Выполняем group by по гранулярностям представленным ниже**

**Столбцы, которые не группируются становятся list'ами**

In [3]:
accuracy_granularity = [
    # "period",
    "rps",
    # "holding_name",
    # "sender_department_name",
    # "recipient_department_name",
    
    "client_sap_id",
    "freight_id",
    "sender_station_id",
    "recipient_station_id",
    "sender_organisation_id",
    "podrod",
    "filial",
]

In [4]:
data_grouped = data.groupby(accuracy_granularity).agg(list)
data_grouped

period  \
rps client_sap_id freight_id sender_station_id recipient_station_id sender_organisation_id podrod filial                             
0   -1            0          2977              38567                7152                   0      2                   [2019-06-01]   
                             3801              26930                7152                   1      2                   [2018-11-01]   
                             3845              28207                8535                   1      2                   [2018-09-01]   
                             4602              31057                0                      0      2                   [2017-02-01]   
                             4871              25250                0                      1      2       [2017-09-01, 2017-10-01]   
...                                                                                                                            ...   
1    2415         2997       13005             33625                10441                  5      2       [2018-04-01, 2018-06-01]   
                                               37870                10441                  5      2       [2018-05-01, 2018-06-01]   
                  3471       32800             12142                7178                   5      2                   [2019-03-01]   
                                               12840                7178                   5      2                   [2018-11-01]   
     2416         1691       27096             27484                10579                  5      2       [2021-04-01, 2021-05-01]   

                                                                                                             real_weight  \
rps client_sap_id freight_id sender_station_id recipient_station_id sender_organisation_id podrod filial                   
0   -1            0          2977              38567                7152                   0      2               [31.0]   
                             3801              26930                7152                   1      2               [90.0]   
                             3845              28207                8535                   1      2                [4.0]   
                             4602              31057                0                      0      2               [40.0]   
                             4871              25250                0                      1      2         [68.0, 68.0]   
...                                                                                                                  ...   
1    2415         2997       13005             33625                10441                  5      2       [136.0, 138.0]   
                                               37870                10441                  5      2        [70.0, 137.0]   
                  3471       32800             12142                7178                   5      2               [67.0]   
                                               12840                7178                   5      2               [67.0]   
     2416         1691       27096             27484                10579                  5      2       [130.0, 130.0]   

                                                                                                         real_wagon_count  
rps client_sap_id freight_id sender_station_id recipient_station_id sender_organisation_id podrod filial                   
0   -1            0          2977              38567                7152                   0      2                   [1]  
                             3801              26930                7152                   1      2                   [4]  
                             3845              28207                8535                   1      2                   [1]  
                             4602              31057                0                      0      2                   [2]  
                        

**Проверяем, что после группировки колонка период содержит только уникальные значения**

In [5]:
for period in data_grouped["period"].tolist():
    if len(set(period)) != len(period):
        raise

**Отбираем из сгруппированной таблицы только те примеры, которые имеют в period '2023-03-01'**

**Только на их грануляциях мы будем делать предсказания**

In [6]:
data_grouped_predict = data_grouped[data_grouped["period"].apply(max) >= data["period"].max()]

**Заменяем существующие периоды на периоды, которые будем предсказывать**

In [7]:
dates = [datetime.datetime.strptime("2022-11-01", "%Y-%m-%d"),
         datetime.datetime.strptime("2022-12-01", "%Y-%m-%d"),
         datetime.datetime.strptime("2023-01-01", "%Y-%m-%d"),
         datetime.datetime.strptime("2023-02-01", "%Y-%m-%d"),
         datetime.datetime.strptime("2023-03-01", "%Y-%m-%d")
]

data_grouped_predict["period"] = [dates.copy() for _ in range(data_grouped_predict.shape[0])]
data_grouped_predict

/var/folders/ww/n61vmv4529s__kvnf2d_7x_jlkmr7f/T/ipykernel_17888/2935264513.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_grouped_predict["period"] = [dates.copy() for _ in range(data_grouped_predict.shape[0])]


period  \
rps client_sap_id freight_id sender_station_id recipient_station_id sender_organisation_id podrod filial                                                      
0   -1            3          10165             30710                26119                  0      1       [2022-11-01 00:00:00, 2022-12-01 00:00:00, 202...   
                                               30807                26119                  0      1       [2022-11-01 00:00:00, 2022-12-01 00:00:00, 202...   
                                               30892                26119                  0      1       [2022-11-01 00:00:00, 2022-12-01 00:00:00, 202...   
                                               31398                26119                  0      1       [2022-11-01 00:00:00, 2022-12-01 00:00:00, 202...   
                                               34452                26119                  0      1       [2022-11-01 00:00:00, 2022-12-01 00:00:00, 202...   
...                                                                                                                                                     ...   
1    2410         1682       32810             36555                28657                  5      1       [2022-11-01 00:00:00, 2022-12-01 00:00:00, 202...   
                                               36988                28657                  5      1       [2022-11-01 00:00:00, 2022-12-01 00:00:00, 202...   
                  1691       32810             11445                25813                  5      2       [2022-11-01 00:00:00, 2022-12-01 00:00:00, 202...   
                                               27374                25813                  4      2       [2022-11-01 00:00:00, 2022-12-01 00:00:00, 202...   
                  1705       32810             37493                28657                  5      1       [2022-11-01 00:00:00, 2022-12-01 00:00:00, 202...   

                                                                                                                         real_weight  \
rps client_sap_id freight_id sender_station_id recipient_station_id sender_organisation_id podrod filial                               
0   -1            3          10165             30710                26119                  0      1                     [62.0, 60.0]   
                                               30807                26119                  0      1        [50.0, 60.0, 120.0, 50.0]   
                                               30892                26119                  0      1                           [40.0]   
                                               31398                26119                  0      1                     [65.0, 60.0]   
                                               34452                26119                  0      1       [105.0, 50.0, 60.0, 230.0]   
...                                                                                                                              ...   
1    2410         1682       32810             36555                28657                  5      1                           [55.0]   
                                               36988                28657                  5      1                           [54.0]   
                  1691       32810             11445                25813                  5      2       [140.0, 70.0, 140.0, 70.0]   
                                               27374                25813                  4      2                    [136.0, 70.0]   
                  1705       32810             37493                28657                  5      1                           [55.0]   

                                                                                                         real_wagon_count  
rps client_sap_id freight_id sender_station_id recipient_station_id sender_organisation_id podrod filial                   
0   -1            3          10165             30710            

**Разварачиваем табличку выше по периоду (flatten by)**

In [8]:
data_future_months = data_grouped_predict.explode(["period"]).reset_index()

### Предсказание

**Применение эвристики**

**Мы выдаем последним (min(12, len(real_wagon_count)) - 1) записям вес равный единице, а последней записи выдаем вес = (min(12, len(real_wagon_count)) + 1)**

In [9]:
data_future_months["forecast_wagon_count"] = (data_future_months["real_wagon_count"].apply(lambda x: sum(x[-12:]) / len(x[-12:])) + data_future_months["real_wagon_count"].apply(lambda x: x[-1])) / 2
data_future_months["forecast_weight"] = (data_future_months["real_weight"].apply(lambda x: sum(x[-12:]) / len(x[-12:])) + data_future_months["real_weight"].apply(lambda x: x[-1])) / 2
data_future_months = data_future_months.drop(columns=["real_wagon_count", "real_weight"])
data_future_months

,rps,client_sap_id,freight_id,sender_station_id,recipient_station_id,sender_organisation_id,podrod,filial,period,forecast_wagon_count,forecast_weight
0,0,-1,3,10165,30710,26119,0,1,2022-11-01,1.0,60.5
1,0,-1,3,10165,30710,26119,0,1,2022-12-01,1.0,60.5
2,0,-1,3,10165,30710,26119,0,1,2023-01-01,1.0,60.5
3,0,-1,3,10165,30710,26119,0,1,2023-02-01,1.0,60.5
4,0,-1,3,10165,30710,26119,0,1,2023-03-01,1.0,60.5
...,...,...,...,...,...,...,...,...,...,...,...
239600,1,2410,1705,32810,37493,28657,5,1,2022-11-01,1.0,55.0
239601,1,2410,1705,32810,37493,28657,5,1,2022-12-01,1.0,55.0
239602,1,2410,1705,32810,37493,28657,5,1,2023-01-01,1.0,55.0
239603,1,2410,1705,32810,37493,28657,5,1,2023-02-01,1.0,55.0


### Сохранение предсказания

In [10]:
example = pd.read_csv("./data/forecast_example.csv", sep=";", decimal=",", encoding="windows-1251")

submission = data_future_months[example.columns]
submission.to_csv("./submission/submission_0.csv", sep=";", decimal=",", encoding="windows-1251", index=False)